# Behavior, preprocessing and motion correction

Welcome to the CaImAn processing pipeline! With these notebooks you can perform all necessary established processing and analysis steps in pre-formatted, easy-to-use chunks. 

Currently these steps are:
 1. **Align and validate behavior**: Align the different behavioral files produces by LabView. Then plot licking and running data over time to remove trials with faulty recording.
 1. **Preprocessing and motion correction**: Take the raw .tif file, do some preprocessing and perform motion correction built into CaImAn. The results are saved as one big .mmap file per recording session.
 2. **Source extraction**: Take the .mmap file and use CaImAn to extract ROIs (putative neurons) and their calcium data from the movie. Results are saved in a CNM object HDF5 file.
 3. **Component evaluation**: Load the CNM object and perform component evaluation. This does a quality check and only keeps ROIs that are actually active neurons. Results are saved by updating the cnm.hdf5 file
 4. **Place cell detection**: Load the finished CNM object and perform the first established step of "true" analysis: finding place cells. The results (including the cnm object) is saved in a new `pcf_results.pickle` file.

## Align and validate behavior

The first thing you want to do after recording a session is to validate that the behavior was recorded properly. For this you first run the `align_behavior()` script that combines the different files produced by LabView. Then you can plot the licking and speed data across time, not aligned to the VR position (for now we only care about the raw behavior). If e.g. the licking sensor was permanently activated and lickings might have been missed, its better to delete the trial now before processing rather than let the faulty recording corrupt the performance analysis.

In [5]:
import sys
sys.path.append('../custom scripts/')

import standard_pipeline.behavior_import as behavior
import standard_pipeline.performance_check as performance

root = r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch6'
%matplotlib qt

In [6]:
# First, we merge the behavioral files into one. If performance_check=True, the licking/stopping ratio is saved
behavior.align_behavior(root, performance_check=True, verbose=False, overwrite=False)


Start processing session W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch6\M71\20210208...
Done!


Start processing session W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch6\M71\20210209...
Done!


Start processing session W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch6\M71\20210210...
Done!


Start processing session W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch6\M71\20210211...
Done!


Start processing session W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch6\M71\20210212...
Done!


Start processing session W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch6\M71\20210214...
Done!


Start processing session W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch6\M72\20210208...
Done!


Start processing session W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch6\M72\20210209...
Done!


Start processing session W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch6\M72\20210210...
Trial incomplete, please remove file!
Skipped trial W

In [7]:
# Now check if any trials show bad/faulty behavior recording (e.g. constantly activated licking sensor)
# Do this for each mouse/session separately. Manually remove bad trials from the session folder
%matplotlib qt
performance.quick_screen_session(r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch5\M62\20210211')

In [11]:
# After filtering out bad trials, you can load the performance data and plot it
# DOES NOT WORK IN NOTEBOOK?
path = [r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch4\M57',
       r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch4\M59',
       r'W:\Neurophysiology-Storage1\Wahl\Hendrik\PhD\Data\Batch4\M60']
stroke = ['M2, M7']
data = performance.load_performance_data(roots=path,norm_date='20201209',
                                         stroke=stroke)
performance.plot_all_mice_separately(data, field='licking_binned', rotate_labels=False, scale=1.75, vlines=[14.5], columns=1)

## Preprocessing and motion correction

The first step we want to do is preprocess our raw .tif files. There are two problems with the .tif files produced by Scanimage that we want to correct before proceeding with the actual processing.


First, even and odd lines can be (slightly) misaligned if the settings of the microscope have not been set properly before the session. This can be corrected by calculating the crosscorrelation between even and odd lines and shifting them by the number of pixels that maximise the correlation.


Second, the pixel values coming from the microscope have arbitrary units, which is why we later transform the values into a normalized form which is comparable between neurons, sessions and microscopes. Thus, the absolute pixel intensity is not really relevant for later analysis. However, the Scientifica microscope that we are using to acquire the data scales the data so that the noise distribution is centered on 0. This causes up to half of the values (at least without calcium transients) to actually be negative. This is a problem because CaImAn uses a model called **non-negative** matrix factorization, which assumes that all values are positive (which kind of makes sense, you cannot have "negative" fluorescence). This can be corrected by shifting all pixel values by a certain amount to eliminate negative values.

After this is done, CaImAn performs motion correction and saves the result as a mmap file in the session folder.

In [1]:
# Import packages
import sys
sys.path.append('../custom scripts/')

import standard_pipeline.place_cell_pipeline as pipe
import standard_pipeline.behavior_import as behavior
import matplotlib.pyplot as plt
import caiman as cm
from caiman.source_extraction import cnmf
import numpy as np

# Start cluster for parallel processing
c, dview, n_processes = cm.cluster.setup_cluster(backend='local', n_processes=None, single_thread=False)


C:\Anaconda3\envs\caiman\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\caiman\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\caiman\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\caiman\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
## Set parameters

root = r'W:\Neurophysiology-Storage1\Wahl\Jithin\Imaging\Batch_4\M35\Pre_Stroke\Caudal\50um'

# dataset dependent parameters
fr = 30  # imaging rate in frames per second
decay_time = 0.4  # length of a typical transient in seconds (0.4)
dxy = (0.83, 0.76)   # spatial resolution in x and y in (um per pixel) [(1.66, 1.52) for 1x, (0.83, 0.76) for 2x]
max_shift_um = (70., 70.)  # maximum shift in um
patch_motion_um = (250., 250.)  # patch size for non-rigid correction in um, default (250,250)

# motion correction parameters
pw_rigid = True  # flag to select rigid vs pw_rigid motion correction
# maximum allowed rigid shift in pixels
max_shifts = [int(a / b) for a, b in zip(max_shift_um, dxy)]
# start a new patch for pw-rigid motion correction every x pixels
strides = tuple([int(a / b) for a, b in zip(patch_motion_um, dxy)])
# overlap between patches (size of patch in pixels: strides+overlaps)
overlaps = (100, 100)   #default (12,12)
# maximum deviation allowed for patch with respect to rigid shifts
max_deviation_rigid = 3
# number of iterations for motion correction (set to 2 for strong artifacts)
niter_rig = 2

mc_dict = {
    'fnames': None,
    'fr': fr,
    'decay_time': decay_time,
    'dxy': dxy,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': 'copy',
    'n_processes': n_processes,
    'niter_rig': niter_rig
}

opts = cnmf.params.CNMFParams(params_dict=mc_dict)

In [ ]:
motion_file, dview = pipe.motion_correction(root, opts, dview, basename = 'Caudal_recording', percentile=0.01, get_images=True, overwrite=False)

In [3]:
# To check the motion-corrected movie, we can save it as a TIFF file and open it in ImageJ
pipe.export_tif(root)

Loading file W:\Neurophysiology-Storage1\Wahl\Jithin\Imaging\Batch_4\M35\Pre_Stroke\Caudal\50um\memmap__d1_512_d2_472_d3_1_order_C_frames_20000_.mmap...
Start saving memmap movie to TIFF...
Done!
